In [ ]:
!pip install tf_keras_vis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.2 MB/s eta 0:00:00


In [ ]:
# %reload_ext autoreload
# %autoreload 2

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import tensorflow as tf
from tf_keras_vis.utils import num_of_gpus

_, gpus = num_of_gpus()
print('Tensorflow recognized {} GPUs'.format(gpus))

Tensorflow recognized 1 GPUs


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image

In [ ]:
import os
train_dir = '/content/drive/MyDrive/Kaggle3/Test_Alphabet'
categories = os.listdir(train_dir)
len(categories)
# length of training images
length = 0
train_images_names_and_paths = {}

for cat in categories:
    train_images_names_and_paths[cat] = os.listdir(train_dir + '/' + cat)
    length += len(os.listdir(train_dir + '/' + cat))

print('Total Length of Training Images:', length)

Total Length of Training Images: 2700


In [ ]:
os.listdir(train_dir)

['A',
 'B',
 'Blank',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications import VGG19
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import seaborn as sns
from keras.preprocessing.image import img_to_array, array_to_img
from keras.applications.vgg19 import preprocess_input


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/content/drive/MyDrive/Kaggle3/Train_Alphabet/'
test_data_dir = '/content/drive/MyDrive/Kaggle3/Test_Alphabet/'

# Initialize ImageDataGenerator for train and test data
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(48, 48),
        batch_size=32,
        class_mode='categorical')

# Flow validation images in batches of 32 using test_datagen generator
test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(48, 48),
        batch_size=32,
        class_mode='categorical')

# Now you can use train_generator and test_generator for training and testing your model


Found 24301 images belonging to 27 classes.
Found 2700 images belonging to 27 classes.


In [ ]:

from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

# Load the VGG19 model without the top layer
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Freeze the base model layers
base_model.trainable = False

# Add our own classification layers on top
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
predictions = Dense(27, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples // train_generator.batch_size,
      epochs=12,
      validation_data=test_generator,
      validation_steps=test_generator.samples // test_generator.batch_size)

In [ ]:
exp_model_path = '/content/drive/MyDrive/my_model.h5'

# Save the model
model.save(exp_model_path)

In [ ]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('/content/drive/MyDrive/my_model.h5')


In [ ]:
model_path = '/content/drive/MyDrive/exp_model.keras'

model.save(model_path)

In [ ]:
model

In [ ]:
model.summary

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img

# Folder containing the images
test_data_dir = '/content/drive/MyDrive/Kaggle3/Test_Alphabet/'

# Get the list of subdirectories (folders) inside test_data_dir
subdirectories = [subdir for subdir in os.listdir(test_data_dir) if os.path.isdir(os.path.join(test_data_dir, subdir))]

# Load one image from each subdirectory
images = []
image_titles = []
for subdir in subdirectories:
    subdir_path = os.path.join(test_data_dir, subdir)
    image_filenames = [filename for filename in os.listdir(subdir_path) if filename.endswith(('.png', '.jpg', '.jpeg'))]
    if image_filenames:
        # Pick the first image in the directory
        filename = image_filenames[0]
        img = load_img(os.path.join(subdir_path, filename), target_size=(48, 48))
        images.append(np.array(img))
        image_titles.append(subdir)

# Plotting
num_images = len(images)
fig, ax = plt.subplots(num_images, 1, figsize=(6, 6*num_images))

for i, (img, title) in enumerate(zip(images, image_titles)):
    ax[i].imshow(img)
    ax[i].set_title(title, fontsize=12)
    ax[i].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear

replace2linear = ReplaceToLinear()

In [ ]:
images[0].shape

In [ ]:
from tf_keras_vis.utils.scores import CategoricalScore

# Create an empty list to store the scores for each image
scores = []

for img in images:
    # Reshape the image to match the model's input shape if needed
    # img = np.expand_dims(img, axis=0)  # Uncomment if needed

    # Compute the prediction probabilities for the image using the model
    pred_probs = model.predict(img.reshape(1, 48, 48, 3))  # Assuming the input shape is (48, 48, 3)

    # Extract the index of the class with the highest probability
    pred_class_index = np.argmax(pred_probs)

    # Append the index to the scores list
    scores.append(pred_class_index)

# Convert the list of scores to a CategoricalScore object
score = CategoricalScore(scores)


In [ ]:
from matplotlib import pyplot as plt
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore
import numpy as np

# Create Gradcam object
replace2linear = ReplaceToLinear()
gradcam = Gradcam(model, model_modifier=replace2linear, clone=True)

# Create an empty list to store the heatmaps
heatmaps = []

# Iterate over each image
for img in images:
    # Compute the prediction probabilities for the image using the model
    pred_probs = model.predict(np.expand_dims(img, axis=0))
    # Extract the index of the class with the highest probability
    pred_class_index = np.argmax(pred_probs)
    # Create a score object with the predicted class index
    score = CategoricalScore([pred_class_index])
    # Generate heatmap with GradCAM for the current image
    cam = gradcam(score, img, penultimate_layer=-1)
    # Append the heatmap to the heatmaps list
    heatmaps.append(cam)

# Render images with heatmaps
fig, axes = plt.subplots(1, len(images), figsize=(15, 5))
for i, (img, cam) in enumerate(zip(images, heatmaps)):
    heatmap = np.uint8(cm.jet(cam)[..., :3] * 25)
    axes[i].imshow(img)
    axes[i].imshow(heatmap, cmap='jet', alpha=0.5) # overlay
    axes[i].axis('off')
plt.show()


In [ ]:
import numpy as np

# Create an empty list to store the predicted class indices for each image
predicted_classes = []

# Iterate over each image
for img in images:
    # Reshape the image to match the model's input shape if needed
    # img = np.expand_dims(img, axis=0)  # Uncomment if needed

    # Compute the prediction probabilities for the image using the model
    pred_probs = model.predict(img.reshape(1, 48, 48, 3))  # Assuming the input shape is (48, 48, 3)

    # Extract the index of the class with the highest probability
    pred_class_index = np.argmax(pred_probs)

    # Append the predicted class index to the list
    predicted_classes.append(pred_class_index)

# Print the predicted class indices
print("Predicted class indices:")
print(predicted_classes)

# Print the corresponding class labels
class_labels = [image_titles[i] for i in predicted_classes]
print("\nCorresponding class labels:")
print(class_labels)


In [ ]:
print(image_titles)

In [ ]:
!pip install vis

In [ ]:
import keras
import vis

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
_img = load_img('/content/drive/MyDrive/Kaggle3/Test_Alphabet/A/test.png', target_size=(48,48))
plt.imshow(_img)
plt.show()

In [ ]:
img = img_to_array(_img)
img = preprocess_input(img)
y_pred = model.predict(img[np.newaxis,...])
class_idx_sorted = np.argsort(y_pred.flatten())[::-1]

In [ ]:
import tensorflow as tf

In [ ]:
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear

replace2linear = ReplaceToLinear()
gradcam = Gradcam(model, model_modifier=replace2linear, clone=True)

# Create an empty list to store the heatmaps
heatmaps = []

# Iterate over each image
for img in images:
    # Preprocess the image
    img = tf.cast(img, tf.float32) / 255.0

    # Compute the prediction probabilities for the image using the model
    pred_probs = model.predict(np.expand_dims(img, axis=0))
    # Extract the index of the class with the highest probability
    pred_class_index = np.argmax(pred_probs)
    # Create a score object with the predicted class index
    score = CategoricalScore([pred_class_index])
    # Generate heatmap with GradCAM for the current image
    try:
        cam = gradcam(score, img, penultimate_layer=-1)
    except Exception as e:
        print(f"Error generating heatmap for image: {e}")
        continue
    # Append the heatmap to the heatmaps list
    heatmaps.append(cam)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Render images with heatmaps
fig, axes = plt.subplots(1, len(images), figsize=(15, 5))
for i, (img, cam) in enumerate(zip(images, heatmaps)):
    heatmap = np.uint8(cm.jet(cam)[..., :3] * 25)
    heatmap = heatmap.squeeze()  # Remove the first dimension
    axes[i].imshow(img)
    axes[i].imshow(heatmap, cmap='jet', alpha=0.5)  # overlay
    axes[i].axis('off')
plt.show()

In [ ]:
# Render images with heatmaps
fig, axes = plt.subplots(1, len(images), figsize=(15, 5))
for i, (img, cam) in enumerate(zip(images, heatmaps)):
    # Convert the image to BGR format for OpenCV
    img_bgr = cv2.cvtColor(np.uint8(img), cv2.COLOR_RGB2BGR)

    # Generate the heatmap for the current image
    heatmap = np.uint8(cm.jet(cam)[..., :3] * 25)
    heatmap = heatmap.squeeze()  # Remove the first dimension

    # Overlay the heatmap on the original image
    overlay_img = cv2.addWeighted(img_bgr, 0.7, cv2.applyColorMap(heatmap, cv2.COLORMAP_JET), 0.3, 0)

    # Display the image with the heatmap
    axes[i].imshow(cv2.cvtColor(overlay_img, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for displaying with matplotlib
    axes[i].axis('off')

plt.show()

In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg19 import preprocess_input

def get_heatmap(model, block5_conv4, image):
    # Expand dimensions to create a batch of 1 image
    image = np.expand_dims(image, axis=0)

    # Preprocess the image
    processed_image = preprocess_input(image)

    # Define the gradient model
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(block5_conv4).output, model.output])

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(processed_image)
        class_idx = tf.argmax(preds[0])
        class_output = preds[:, class_idx]
        grads = tape.gradient(class_output, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer_output), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    return heatmap

def overlay_heatmap(heatmap, image):
    heatmap = np.uint8(255 * heatmap)
    heatmap = tf.keras.preprocessing.image.array_to_img(heatmap)
    heatmap = heatmap.resize((image.shape[1], image.shape[0]))
    heatmap = tf.keras.preprocessing.image.img_to_array(heatmap)

    overlayed_img = heatmap * 0.4 + image

    return overlayed_img / np.max(overlayed_img)



# Name of the last convolutional layer
last_conv_layer_name = 'block5_conv4'  # Change this according to your model architecture

# Load and preprocess the images
test_data_dir = '/content/drive/MyDrive/Kaggle3/Test_Alphabet/'
subdirectories = [subdir for subdir in os.listdir(test_data_dir) if os.path.isdir(os.path.join(test_data_dir, subdir))]

for subdir in subdirectories:
    subdir_path = os.path.join(test_data_dir, subdir)
    image_filenames = [filename for filename in os.listdir(subdir_path) if filename.endswith(('.png', '.jpg', '.jpeg'))]
    if image_filenames:
        # Load the image
        filename = image_filenames[0]
        img = load_img(os.path.join(subdir_path, filename), target_size=(48, 48))
        img_array = img_to_array(img)

        # Get the heatmap
        heatmap = get_heatmap(model, last_conv_layer_name, img_array)

        # Overlay heatmap on the image
        overlayed_img = overlay_heatmap(heatmap, img_array)

        # Plot the original image, heatmap, and overlayed image
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 3, 1)
        plt.imshow(img)
        plt.title('Original Image')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(heatmap)
        plt.title('Heatmap')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(overlayed_img)
        plt.title('Overlayed Image')
        plt.axis('off')

        plt.show()
